In [21]:
# Example of simple model load and evaluate

# ===============LICENSE_START=======================================================
# Apache-2.0
# ===================================================================================
# Copyright (C) 2019 AT&T Intellectual Property  All rights reserved.
# ===================================================================================
# This software file is distributed by AT&T
# under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# This file is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ===============LICENSE_END=========================================================


In [29]:
import numpy as np
import pandas as pd
import os,sys,shutil  # file checks
import dill as pickle   # serialize functions and data as compressed binary 
import gzip  # compression 
import yaml   # configuration file
import time  # time tracking

from acumos.modeling import Model, List, Dict, create_namedtuple, create_dataframe
from acumos.wrapped import load_model, _pack_pb_msg
from google.protobuf.json_format import MessageToJson, MessageToDict
from sklearn.ensemble import RandomForestClassifier

from acumos.modeling import Model, List, Dict, create_namedtuple, create_dataframe
from acumos.session import AcumosSession, Requirements

import util_call

# load our configutaion
config_path = 'config.yaml'
if not os.path.isfile(config_path):
    print("Sorry, can't find the configuration file {}, aborting.".format(config_path))
    sys.exit(-1)
config = yaml.safe_load(open(config_path))

In [30]:
clf = RandomForestClassifier(random_state=0, n_estimators=100)
clf.fit(np.random.rand(10,2), np.random.randint(1, size=10))

# for now we'll make a temporary text frame that mimics the expected "signature"
TextFrame = create_namedtuple('TextFrame', [('text',str)])
def _dummy_sentiment_class(df: TextFrame) -> str:
    '''Returns a sentiment classification classifications'''
    _ = clf.predict(np.random.rand(1,2))
    return "dummy"
def _dummy_sentiment_score(df: TextFrame) -> float:
    '''Returns a sentiment classification classifications'''
    _ = clf.predict(np.random.rand(1,2))
    return float(0)
# create a model example here and dump it to disk
model = Model(sent_classify=_dummy_sentiment_class, sent_predict=_dummy_sentiment_score)

# create model so that we can run it locally
session = AcumosSession()
path_dump = os.path.join('data', config["publish"]["name_model3"])
if os.path.exists(path_dump):
    shutil.rmtree(path_dump)
session.dump(model, config["publish"]["name_model3"], 'data')  # creates ~/<name_publish>

# load model from disk, see that it is a nicely "wrapped" model
wrapped_model = load_model(config["publish"]["name_model3"])
print(wrapped_model)

AcumosError: Provided path model3-setiment-ex does not contain an Acumos model

In [10]:
# actual evaluation code...

# test 1 - take an example of our own sample (where we can see format from the model)
X = [[[0,0], "I've had better", "not much", int(time.time()), ["office products"], "rubbery item"]]
url_computed = "TOBEDEFINED"
df_score = util_call.score_model(wrapped_model, X, url_computed)



NameError: name 'wrapped_model' is not defined

In [50]:
# test 2 - evaluate our test data

# read our larger datasets as binary files
with gzip.open(config["path"]["etl"], 'rb') as f:
    df = pickle.load(f)
df_score = util_call.score_model(wrapped_model, df["X_test"].values, False)

# TODO: save new scores with unique row ID for TRAINING data
# TODO: save new scores with unique row ID for TEST data

ReviewDataFrame(helpful=[1, 1], reviewText="There's nothing more dispiriting than spending hours in highlighting and tabbing the important parts of a document with tape flags and then finding, after filing the document away, that some of the flags have fallen off or are folded into the document, no longer drawing attention to the sections carefully identified.  Using these Post-It durable tabs will prevent this from happening, and also permit writing on the tab flagging the page with a marker.  Another benefit is that these tabs adhere well enough to allow a document which has already been reviewed and tabbed to be run through a photocopy machine's feeder if a copy is needed without the tabs coming off in the feeder mechanism.", summary='These tabs perform as promised', unixReviewTime=1293840000, categories=['office products', 'office & school supplies', 'labels, indexes & stamps', 'index tabs & tab inserts', 'tab inserts'], description='Post-it(R) Tabs in 1 inch size create necessary 

TypeError: tuple indices must be integers or slices, not str